<a href="https://colab.research.google.com/github/maximoscortes/maximoscortes/blob/main/andrexversor_bilstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

epocas = 10

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow import keras
import librosa
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
from tqdm import tqdm
import os
from pathlib import Path
import random
import sys

from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional

import time
import json
from json import JSONEncoder
!pip install CherryPy #webserver package
!pip install cherrypy_cors



# Frequency smoothing
def freqSmooth(x, sm=1.0/24.0):
    s = sm if sm > 1.0 else np.sqrt(2.0**sm)
    N = len(x)
    y = np.zeros_like(x)
    for i in range(N):
        i1 = max(int(np.floor(i/s)), 0)
        i2 = min(int(np.floor(i*s)+1), N-1)
        if i2 > i1:
            y[i] = np.mean(x[i1:i2])
    return y

# FFT with frequency smoothing
def plot_fft(x, fs, sm=1.0/24.0):
    fft = freqSmooth(20 * np.log10(np.abs(np.fft.rfft(x) + 1.0e-9)), sm=sm)
    freqs = np.fft.rfftfreq(len(x), 1.0 / fs)
    return freqs, fft

# load file from FMA dataset
def load_fma_file(files, filepath, fs, N):
    subfolder = files[random.randrange(0, len(files))]
    subfolderpath = filepath + subfolder + '/'
    subfiles = os.listdir(subfolderpath)
    
    mp3path = subfolderpath + subfiles[random.randrange(0, len(subfiles))]
    x, sr = librosa.load(mp3path, sr=fs, mono=True)
        
    start_idx = int(random.uniform(0, len(x) - N))
    return x[start_idx:start_idx+N]

def plot_metric(history, key, val=True):
    loss = history.history[key]
    if val:
        val_loss = history.history['val_' + key]
    epochs = range(1, len(loss) + 1)

    plt.plot(epochs, loss, 'g.', label='Training ' + key)
    if val:
        plt.plot(epochs, val_loss, 'b', label='Validation ' + key)
    plt.title('Training and validation ' + key)
    plt.xlabel('Epochs')
    plt.ylabel(key)
    plt.legend()


def pre_emphasis_filter(x, coeff=0.85):
    return tf.concat([x[:, 0:1, :], x[:, 1:, :] - coeff*x[:, :-1, :]], axis=1)

def dc_loss(target_y, predicted_y):
    return tf.reduce_mean(tf.square(tf.reduce_mean(target_y) - tf.reduce_mean(predicted_y))) / tf.reduce_mean(tf.square(target_y))

# Error-to-signal loss
def esr_loss(target_y, predicted_y, emphasis_func=lambda x : x):
    target_yp = emphasis_func(target_y)
    pred_yp = emphasis_func(predicted_y)
    return tf.reduce_sum(tf.square(target_yp - pred_yp)) / tf.reduce_sum(tf.square(target_yp))

class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

class Model():
    def __init__(self, loss_func, optimizer=keras.optimizers.Adam()):
        self.model = keras.Sequential()
        self.opt = optimizer
        self.loss_func = loss_func

        self.train_loss = []
        self.train_err = []
        self.val_loss = []
        self.val_err = []

    def run_training(self, in_train, out_train, epoch_loss, epoch_error, N_skip, N_block):
        N_samples = in_train.shape[1]
        self.model.reset_states() # clear existing state
        self.model(in_train[:, :N_skip, :]) # process some samples to build up state

        # iterate over blocks
        for n in range(N_skip, N_samples-N_block, N_block):
            # compute loss
            with tf.GradientTape() as tape:
                y_pred = self.model(in_train[:, n:n+N_block, :])
                loss = self.loss_func(out_train[:, n:n+N_block, :], y_pred)

            # apply gradients
            grads = tape.gradient(loss, self.model.trainable_variables)
            self.opt.apply_gradients(zip(grads, self.model.trainable_variables))

            # update training metrics
            epoch_loss.update_state(loss)
            epoch_error.update_state(out_train[:, n:n+N_block, :], y_pred)

    def run_validation(self, in_val, out_val, val_loss, val_error):
        self.model.reset_states()
        y_val = self.model(in_val)
        loss = self.loss_func(out_val, y_val)
        val_loss.update_state(loss)
        val_error.update_state(out_val, y_val)

    def run_epoch(self, epoch, in_train, out_train, in_val=None, out_val=None, N_skip=1000, N_block=2048):
        start = time.perf_counter()
        epoch_loss_avg = keras.metrics.Mean()
        epoch_error = keras.metrics.MeanSquaredError()
        val_loss_avg = keras.metrics.Mean()
        val_error = keras.metrics.MeanSquaredError()

        self.run_training(in_train, out_train, epoch_loss_avg, epoch_error, N_skip, N_block)
        if in_val is not None:
            self.run_validation(in_val, out_val, val_loss_avg, val_error)
            
        # end epoch
        self.train_loss.append(epoch_loss_avg.result())
        self.train_err.append(epoch_error.result())
        self.val_loss.append(val_loss_avg.result())
        self.val_err.append(val_error.result())
        dur = time.perf_counter() - start

        print("epocaaa {:03d} - tiempo: {:.4f}s, perdida: {:.4f}, error: {:.3%}, perdida de validacion: {:.4f}, Val_Error: {:.3%}".format(            epoch+1, dur, epoch_loss_avg.result(), epoch_error.result(), val_loss_avg.result(), val_error.result()))

    def train_until(self, loss_stop, in_train, out_train, in_val=None, out_val=None, N_skip=1000, N_block=2048):
        epoch = 0
        while True:
            self.run_epoch(epoch, in_train, out_train, in_val, out_val, N_skip, N_block)

            if self.train_loss[-1] < loss_stop:
                break

            epoch += 1

    def train(self, num_epochs, in_train, out_train, in_val=None, out_val=None, N_skip=1000, N_block=2048, save_model=None, save_hist=None):
        for epoch in range(num_epochs):
            self.run_epoch(epoch, in_train, out_train, in_val, out_val, N_skip, N_block)

            if save_model is not None:
                self.save_model(save_model)
            if save_hist is not None:
                self.save_history(save_hist)
        
        print("redi!")


    def plot_loss(self):
        epochs = range(1, len(self.train_loss) + 1)
        plt.plot(epochs, self.train_loss, 'g.', label='Training Loss')
        plt.plot(epochs, self.val_loss, 'b', label='Validation Loss')
        plt.title('Training and validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()

    def plot_error(self):
        epochs = range(1, len(self.train_loss) + 1)
        plt.plot(epochs, self.train_err, 'g.', label='Training Error')
        plt.plot(epochs, self.val_err, 'b', label='Validation Error')
        plt.title('Training and validation Error')
        plt.xlabel('Epochs')
        plt.ylabel('Error')
        plt.legend()

    def save_model_json(self):
        def get_layer_type(layer):
            if isinstance(layer, keras.layers.TimeDistributed):
                return 'time-distributed-dense'

            if isinstance(layer, keras.layers.GRU):
                return 'gru'

            if isinstance(layer, keras.layers.Dense):
                return 'dense'

            if isinstance(layer, keras.layers.LSTM):
                return 'lstm'

            if isinstance(layer, keras.layers.Bidirectional):
                return 'bi'

            return 'unknown'

        def save_layer(layer):
            layercito = {
                "type"    : get_layer_type(layer),
                "shape"   : layer.output_shape,
                "weights" : layer.get_weights()
            }

            return layercito


        model_dict = {}
        model_dict["in_shape"] = self.model.input_shape
        layers = []
        for layer in self.model.layers:
            layersote = save_layer(layer)
            layers.append(layersote)

        model_dict["layers"] = layers
        return model_dict

    def save_model(self, filename):
        model_dict = self.save_model_json()
        with open(filename, 'w') as outfile:
            json.dump(model_dict, outfile, cls=NumpyArrayEncoder)


 

    def load_model_json(self, json,indice):
        in_shape = json["in_shape"][1:]
        self.model.add(keras.layers.InputLayer(input_shape=in_shape))
        with open("headeR.h", "a") as file:
            file.write("\n// Bloque de alturas numero "+str(indice)+" q se hace esto \n")
            file.write("// LSP - 2022\n")
            file.write("estructura_lstm1<8,float>  _pos"+str(indice)+"={")
            n_layer = 0
            for layer in json["layers"]:
                print(in_shape)
                #file.write('int dim_layer'+str(n_layer)+'='+str(in_shape)+';\n')
                weights = layer["weights"]
                np_weights = []
                pesitos= str(weights)
                pesotes = pesitos.replace('[','{')
                pesotes = pesotes.replace(']','}')
                pesotes = pesotes[:-1]
                pesotes = pesotes[1:]
                for w in weights:
                    print(w)
                    np_weights.append(np.array(w))
                
                if layer["type"] == 'time-distributed-dense':
                    file.write('\n// pesos de tiempo distribuido denso numero '+str(n_layer)+'\n'+str(pesotes)+',')
                    d_layer = keras.layers.Dense(layer["shape"][-1], activation='tanh')
                    d_layer.build(input_shape=in_shape)
                    d_layer.set_weights(np_weights)
                    m_layer = keras.layers.TimeDistributed(d_layer)

                elif layer["type"] == 'gru':
                    file.write('\n// gru numero '+str(n_layer)+'\n'+str(pesotes)+',')
                    m_layer = keras.layers.GRU(units=layer["shape"][-1], return_sequences=True)
                    m_layer.build(input_shape=(None, None, in_shape[-1]))
                    m_layer.set_weights(np_weights)

                elif layer["type"] == 'lstm':
                    file.write('\n// pesos de LSTM '+str(n_layer)+'\n'+str(pesotes)+',')
                    m_layer = keras.layers.LSTM(units=layer["shape"][-1], activation="tanh", return_sequences=True, recurrent_activation="sigmoid", use_bias=True, kernel_initializer="glorot_uniform", recurrent_initializer="orthogonal", bias_initializer="random_normal", unit_forget_bias=False)
                    m_layer.build(input_shape=(None, None, in_shape[-1]))
                    m_layer.set_weights(np_weights)

                elif layer["type"] == 'bi':
                    file.write('\n// pesos de bi-LSTM '+str(n_layer)+'\n'+str(pesotes)+',')
                    m_layer = keras.layers.Bidirectional(LSTM(units=layer["shape"][-1], activation="tanh", return_sequences=True, recurrent_activation="sigmoid", use_bias=True, kernel_initializer="glorot_uniform", recurrent_initializer="orthogonal", bias_initializer="random_normal", unit_forget_bias=False), merge_mode='sum' )
                    m_layer.build(input_shape=(None, None, in_shape[-1]))
                    m_layer.set_weights(np_weights)

                
                elif layer["type"] == 'dense':
                    file.write('\n// pesos de denso normal numero '+str(n_layer)+'\n'+str(pesotes) +'};')                   
                    m_layer = keras.layers.Dense(layer["shape"][-1])
                    m_layer.build(input_shape=in_shape)
                    m_layer.set_weights(np_weights)

                else:
                    continue
                
                in_shape = tuple(layer["shape"])
                self.model.add(m_layer)
                n_layer = n_layer +1
      
    def load_model(self, filename,indice):
        with open(filename, 'r') as json_file:
            model_json = json.load(json_file)
        self.load_model_json(model_json,indice)

    def save_history(self, filename):
        history = np.array([self.train_loss, self.train_err, self.val_loss, self.val_err])
        np.savetxt(filename, history)

    def load_history(self, filename):
        history = np.loadtxt(filename)
        self.train_loss = history[0].tolist()
        self.train_err = history[1].tolist()
        self.val_loss = history[2].tolist()
        self.val_err = history[3].tolist()




def model_loss(target_y, predicted_y):
    return esr_loss(target_y, predicted_y, pre_emphasis_filter) + dc_loss(target_y, predicted_y)


def diplernin(indice):
    mish = str(indice)
    pathlist = Path('limpios'+mish).glob('*.wav')
    clean_data = []
    dist_data = []
    NUM_SAMPLES = 20000

    for p in tqdm(pathlist):
        fs = 48000 #np.random.uniform(48000, 96000)
        x, sr = librosa.load(str(p), sr=fs, mono=True)
        N = len(x)
        for wav_dir in ['susio'+mish+'/']:
            y, sr = librosa.load(wav_dir + p.name, sr=fs, mono=True)
            clean_data.append(x[N//2:N//2+NUM_SAMPLES])
            dist_data.append(y[N//2:N//2+NUM_SAMPLES])
            
    print("test de falla:")
    print(clean_data)
    #print("test de falla 2:"+dist_data)
    clean_data = np.asarray(clean_data)
    dist_data = np.asarray(dist_data)
    NUM_FILES = len(clean_data)
    print("numero de archivos", NUM_FILES)
    NUM_TRAIN = int(NUM_FILES * 0.95)
    NUM_VAL = NUM_FILES - NUM_TRAIN
    NUM_INPUTS = 1
    print("ahora despues de la conversion a np")
    print(clean_data)
    print("num traib:")
    print(NUM_TRAIN)
    x_train, x_val = np.split(clean_data, [NUM_TRAIN])  
    y_train, y_val = np.split(dist_data,  [NUM_TRAIN])

    print("xtrain:")
    print(x_train)
    OUT_train  = np.reshape(y_train, (NUM_TRAIN, NUM_SAMPLES, 1))
    OUT_val    = np.reshape(y_val, (NUM_VAL, NUM_SAMPLES, 1))
    IN_train = np.reshape(x_train, (NUM_TRAIN, NUM_SAMPLES, NUM_INPUTS))
    IN_val   = np.reshape(x_val, (NUM_VAL, NUM_SAMPLES, NUM_INPUTS))

    #print("in train", IN_train)

    print(IN_train.dtype)
    print(OUT_train.dtype)
    
    

    np.save("info"+mish+"/out_train.npy", OUT_train)
    np.save("info"+mish+"/out_val.npy", OUT_val)
    np.save("info"+mish+"/in_train.npy", IN_train)
    np.save("info"+mish+"/in_val.npy", IN_val)

    OUT_train = np.load("info"+mish+"/out_train.npy")
    OUT_val   = np.load("info"+mish+"/out_val.npy")
    IN_train  = np.load("info"+mish+"/in_train.npy")
    IN_val    = np.load("info"+mish+"/in_val.npy")

    NUM_SAMPLES = 20000
    NUM_INPUTS = 1
    
    if not os.path.exists('/content/modelos'):
        os.makedirs('/content/modelos')

          
    model_file = 'modelos/andrex'+mish+'.json'
    model_hist = 'modelos/andrex_history'+mish+'.txt'
    model = Model(model_loss, optimizer=keras.optimizers.Adam(learning_rate=5.0e-4))
    risal = os.path.exists(model_file)
    if risal:
        model.load_model(model_file,indice)
        
    else:
        model.model.add(keras.layers.InputLayer(input_shape=(None, NUM_INPUTS)))
        model.model.add(keras.layers.TimeDistributed(keras.layers.Dense(8, activation='tanh')))
        #model.model.add(keras.layers.Bidirectional(LSTM(8, activation="tanh", return_sequences=True, recurrent_activation="sigmoid", use_bias=True, kernel_initializer="glorot_uniform", recurrent_initializer="orthogonal", bias_initializer="random_normal", unit_forget_bias=False), merge_mode='sum'))
        model.model.add(keras.layers.LSTM (16, activation="tanh", return_sequences=True, recurrent_activation="sigmoid", use_bias=True, kernel_initializer="glorot_uniform", recurrent_initializer="orthogonal", bias_initializer="random_normal", unit_forget_bias=False))
        model.model.add(keras.layers.Dense(1))

    model.model.summary()

    #print("in:",IN_train, "out:",OUT_train, "in:",IN_val, "out:",OUT_val, model_file)
    model.train(epocas, IN_train, OUT_train, IN_val, OUT_val, save_model=model_file, save_hist=model_hist)
    #print(len(model.train_loss))
    #print(model.train_loss[-1])
    #print(model.val_loss[-1])
    idx = 2
    #predictions = model.model.predict(IN_train[idx].reshape(1, NUM_SAMPLES, NUM_INPUTS)).flatten()
    #print(esr_loss(OUT_val, model.model.predict(IN_val)))
    #print(esr_loss(OUT_train, model.model.predict(IN_train)))
    model.save_model(model_file)
    model.save_history(model_hist)
    #files.download('modelos/andrex'+mish+'.json') 
    #files.download('modelos/andrex_history'+mish+'.txt')


     |████████████████████████████████| 419 kB 14.6 MB/s 
     |████████████████████████████████| 104 kB 67.9 MB/s 
  Created wheel for httpagentparser: filename=httpagentparser-1.9.2-py2.py3-none-any.whl size=7555 sha256=440ee8b2cb2c9f8a55a9f6b63d9cd05fed453a4acdaeef2a94810b3e583ed757
  Stored in directory: /root/.cache/pip/wheels/02/d1/18/c7f7e8f375bc3779e61342a891cced69c03254348091c2c1f3
Successfully built httpagentparser


In [ ]:
!pip install ffmpeg-python


"""
To write this piece of code I took inspiration/code from a lot of places.
It was late night, so I'm not sure how much I created or just copied o.O
Here are some of the possible references:
https://blog.addpipe.com/recording-audio-in-the-browser-using-pure-html5-and-minimal-javascript/
https://stackoverflow.com/a/18650249
https://hacks.mozilla.org/2014/06/easy-audio-capture-with-the-mediarecorder-api/
https://air.ghost.io/recording-to-an-audio-file-using-html5-and-js/
https://stackoverflow.com/a/49019356
"""
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
from google.colab import files
import wave
import numpy as np


AUDIO_HTML = """
<head>

Hola ! estamos funcionando perfecto ... sigamos avanzando!!!

<meta http-equiv="allow" content="HEAD,GET,PUT,DELETE,OPTIONS" />
</head>
<body>
<div>
<p>
<select id="audioSource" visible="false" ></select>
</p>   
</div>
</body>
<script>
var opciones = [];
var s = 0;
var cosita = "hola";
function gotDevices(deviceInfos) {
 

  for (let i = 0; i !== deviceInfos.length; ++i) {
    const deviceInfo = deviceInfos[i];
    const option = document.createElement('option');
    option.value = deviceInfo.deviceId;
    
    if (deviceInfo.kind === 'audioinput') {
      //option.text = deviceInfo.label || `microphone ${audioInputSelect.length + 1}`;
      //audioInputSelect.appendChild(option);
      opciones[s]=deviceInfo.deviceId;
      ++s;
      console.log("opcion:",deviceInfo.label);
      if ( s == 1 ) { cosita = deviceInfo.deviceId; }
      cosita = deviceInfo.deviceId;
      //console.log("cosita:",cosita);
    }
  }
  //selectors.forEach((select, selectorIndex) => {
  //  if (Array.prototype.slice.call(select.childNodes).some(n => n.value === values[selectorIndex])) {
      
  //    select.value = values[selectorIndex];
      
  //  }
    
  //});
}


function gotStream(stream) {


gumStream = stream;
  var options = {

    mimeType : 'audio/webm'  };           //  mimeType : 'audio/webm;codecs=pcm'  };       
  
  recorder = new MediaRecorder(stream, options);
  datito = [];
  //recorder.reset();
  recorder.ondataavailable = function(e) { 
    datito.push(e.data);
               
    var url = new URL.createObjectURL(datito);// e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    } };
  recorder.start;

  /*
  window.stream = stream; 
  
  rec = new MediaRecorder(stream);
  rec.ondataavailable = e => {
    audioChunks.push(e.data);
    if (rec.state == "inactive"){
      let blob = new Blob(audioChunks,{type:'audio/wav;codecs=pcm'});
      recordedAudio.src = URL.createObjectURL(blob);
      recordedAudio.controls=true;
      recordedAudio.autoplay=true;
      audioDownload.href = recordedAudio.src;
      audioDownload.download = 'wav';
      audioDownload.innerHTML = 'download';
    }
  }
  */
}

function handleError(error) {
  console.log('navigator.MediaDevices.getUserMedia error: ', error.message, error.name);
}

//function start() {
	// Second call to getUserMedia() with changed device may cause error, so we need to release stream before changing device
//  if (window.stream) {
//  	stream.getAudioTracks()[0].stop();
//  }

//}


//audioInputSelect.onchange = start;





var data;

function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "listoco";
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}
    var my_div = document.createElement("DIV");
    var my_p = document.createElement("P");
    var my_btn = document.createElement("BUTTON");
    var t = document.createTextNode("Press to start recording");

    my_btn.appendChild(t);
    my_div.appendChild(my_btn);
    document.body.appendChild(my_div);

    var base64data = 0;
    var reader;
    var recorder, gumStream;
    var recordButton = my_btn;

 navigator.mediaDevices.enumerateDevices()
.then(gotDevices).then(pipi)
.catch(handleError);

function pipi(){



    var handleSuccess = function(stream) {
      gumStream = stream;
      var options = {
        mimeType : 'audio/webm;codecs=pcm'
      };            
      recorder = new MediaRecorder(stream, options);
      recorder.ondataavailable = function(e) {            
        var url = URL.createObjectURL(e.data);
        var preview = document.createElement('audio');
        preview.controls = true;
        preview.src = url;
        document.body.appendChild(preview);

        reader = new FileReader();
        reader.readAsDataURL(e.data); 
        reader.onloadend = function() {
          base64data = reader.result;
          
        }
      };
      
      recorder.start();
      };



    console.log("cosita:",cosita);
      var audioSource = opciones[7];
      console.log("este es el inputaudioSource",audioSource, " audio ",opciones[6]);
      console.log("estas las opciones",opciones);
      const constraints = {
        audio: {
              autoGainControl: false,
        channelCount: 2,
        echoCancellation: false,
        latency: 0,
        noiseSuppression: false,
        sampleRate: 48000,
        sampleSize: 24,
        volume: 1.0,
        deviceId: audioSource }};
      console.log("ahora estos",constraints);

      navigator.mediaDevices.getUserMedia(constraints).then(handleSuccess).catch(handleError);

    recordButton.innerText = "Recording... press to stop";
}



setTimeout(function () { recordButton.click(); }, 6000); //cantidad de segundos de grabacion !



function pipipi(){
  
    

    data = new Promise(resolve=>{
    recordButton.onclick = () =>{
    toggleRecording();

    sleep(2000).then(() => {

      resolve(base64data.toString())
      
    });

    }
    });
 return data;

}


data = pipipi();
 
   
</script>
"""


def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True) #-ss 00:00:20 -to 00:00:40
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))
  
  return audio, sr

def save_wav_channel(fn, wav, channel):
    '''
    Take Wave_read object as an input and save one of its
    channels into a separate .wav file.
    '''
    # Read data
    nch   = wav.getnchannels()
    depth = wav.getsampwidth()
    wav.setpos(0)
    sdata = wav.readframes(wav.getnframes())

    # Extract channel data (24-bit data not supported)
    typ = { 1: np.uint8, 2: np.uint16, 4: np.uint32 }.get(depth)
    if not typ:
        raise ValueError("sample width {} not supported".format(depth))
    if channel >= nch:
        raise ValueError("cannot extract channel {} out of {}".format(channel+1, nch))
    #print ("Extracting channel {} out of {} channels, {}-bit depth".format(channel+1, nch, depth*8))
    data = np.fromstring(sdata, dtype=typ)
    ch_data = data[channel::nch]

    # Save channel to a separate file
    outwav = wave.open(fn, 'w')
    outwav.setparams(wav.getparams())
    outwav.setnchannels(1)
    outwav.writeframes(ch_data.tostring())
    outwav.close()


In [ ]:
#sampleo y entrenamiento
from os import remove


import cherrypy_cors
from google.colab import files
import cherrypy
numero = 31
import scipy.io.wavfile
from google.colab.output import eval_js
pulentisimo = eval_js("google.colab.kernel.proxyPort(8888)")
print(pulentisimo+'/a')
import subprocess
subprocess.Popen(["python", "/content/test.py", "8888"]) 
f = open("direccion.pan", "w")
f.write(pulentisimo)
f.close()
FINFIN= """ 
<script>location.reload();</script>

"""

import matplotlib.pyplot as plt
import numpy as np
import wave
import sys


while numero < 100:
    mish = str(numero)
    if not os.path.exists('/content/limpios'+mish):
        os.makedirs('/content/limpios'+mish)
    if not os.path.exists('/content/susio'+mish):
        os.makedirs('/content/susio'+mish)
    if not os.path.exists('/content/info'+mish):
        os.makedirs('/content/info'+mish)
    on = 0
    while on < 2:
        audio = []
        f = open("sinimportancia.chu", "w")
        f.write(str(numero))
        f.close()
        mish = str(numero)
        audio, sr = get_audio()  # aqui recolecta el audio de javascript !!!
        filon = "doble"+str(on)+".wav"
        print("filon : "+ filon)
        #audion = audio[sr*2..length(audio)] #for ()
        audion= []
        eee = 0
        print("el coson:"+str(sr*2)+" y el otro "+str(len(audio)))
        #for uuu in range(sr*2,len(audio)):
          #print("eee:"+str(eee)+" uuu:"+str(uuu))
          #audion[eee] = audio[uuu]  
          #eee=eee+1
        #print("aki esta el coso del coso nuevo:"+audio)
        #print("aki esta el coso del coso nuevo 2:"+audion)
        scipy.io.wavfile.write(filon, sr, audio)
        wav = wave.open(filon)
        filote = '/content/limpios'+mish+'/sample'+str(on)+'.wav'
        filotin = '/content/susio'+mish+'/sample'+str(on)+'.wav'
        save_wav_channel('/content/limpios'+mish+'/sample'+str(on)+'.wav', wav, 0)
        save_wav_channel('/content/susio'+mish+'/sample'+str(on)+'.wav', wav, 1)
        print("sample rate de la conversion:",sr)
        remove(filon)
        on = on + 1
        #display(HTML(FINFIN))
        spf = wave.open(filote, "r")
        spf2 = wave.open(filotin, "r")
        # Extract Raw Audio from Wav File
        signal = spf.readframes(-1)
        signal2 = spf2.readframes(-1)
        print(signal)
        signal = np.frombuffer(signal)#, "Int16")
        signal2 = np.frombuffer(signal2)#, "Int16")
      
        #plt.figure(1)
        #plt.subplot(211)
        #plt.plot(signal, color="red")
        #plt.subplot(212)
        #plt.title("Signal Wave...")
        #plt.plot(signal2, color="green")
        #plt.show()
       

    diplernin(numero)
    numero = numero + 1


In [ ]:
#fabricacion de encabezado de c++

archivosjason = Path('modelos').glob('*.json')
if os.path.exists("headeR.h"):
    os.remove("headeR.h")

with open("headeR.h", "a") as file:
    file.write("// este va en el private del punto H\n")
    file.write("std::unique_ptr<modelo_lstm1<8>> _LSTM[100];\n")
    file.write("\n")
    file.write("// este al inicializar\n")
    for i in range(100):
        file.write("_LSTM["+str(i)+"] = std::make_unique<modelo_lstm1<8>>(_pos"+str(i)+");\n")


model = Model(model_loss, optimizer=keras.optimizers.Adam(learning_rate=5.0e-4))
for n in tqdm(archivosjason):
        
        #str(n)
        for wav_dir in ['modelos/']:
            model_file=wav_dir + n.name
            phone_number = model_file
            dd = ''.join([n for n in phone_number if n.isdigit()])
            ddd = int(dd)
            print(phone_number,dd,ddd)
            model.load_model(model_file,ddd)
            #print(model)
            


In [ ]:
#partir wav en pedazitos

!pip install pydub
from pydub import AudioSegment
import math

class SplitWavAudioMubin():
    def __init__(self, folder, filename):
        self.folder = folder
        self.filename = filename
        self.filepath = folder + '/' + filename
        
        self.audio = AudioSegment.from_wav(self.filepath)
    
    def get_duration(self):
        return self.audio.duration_seconds
    
    def single_split(self, from_min, to_min, split_filename):
        t1 = from_min * 60 * 1000
        t2 = to_min * 60 * 1000
        split_audio = self.audio[t1:t2]
        split_audio.export(self.folder + '/' + split_filename, format="wav")
        
    def multiple_split(self, min_per_split):
        total_mins = math.ceil(self.get_duration() / 60)
        for i in range(0, total_mins, min_per_split):
            split_fn = str(i) + '_' + self.filename
            self.single_split(i, i+min_per_split, split_fn)
            #print(str(i) + ' Done')
            if i == total_mins - min_per_split:
                #print('All splited successfully')

folder = '.'
file = 'doble0.wav'
split_wav = SplitWavAudioMubin(folder, file)
split_wav.multiple_split(min_per_split=1)

In [ ]:
#test.py

import cherrypy
import sys
import random
import string
from google.colab.output import eval_js


#print(pulentisimo)
#parser = argparse.ArgumentParser()
#parser.parse_args()

class HelloWorld:
    @cherrypy.expose
    def index(self):
        f = open("sinimportancia.chu", "r")
        return f.read()
    @cherrypy.expose
    def a(self, length=8):
        f = open("direccion.pan", "r")
        direcsaund = f.read()   #aki va la direcsion html
        htttp =  '''
        <!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <p style="color: forestgreen;">
  <title>..::cosito del coso rulezx::..</title>

  en este script se esta cocinando todo lo relacionado con 
  comunicacion desde el compu hacia el arduino
  se envia un codigo correspondiente 
  a la posicion q el robot 
  necesita y el 
  arduino lo 
  hace

</head>
<body>

  <div class="serial-scale-div">
        <button class="btn" id="connect-to-serial">enganchar puerto</button>
  </div>


  <button id="put-serial-messages">manda mensaje hola</button>

  <button id="get-serial-messages">lee mensajes</button>
  
  <div id="serial-messages-container">
    <div class="message"></div>
  </div>

  <script>
var writer;// = port.writable.getWriter();
document.body.style.backgroundColor = "green";
//document.write(str.fontcolor( "green" ));

    "use strict";
    class SerialScaleController {
        constructor() {
            this.encoder = new TextEncoder();
            this.decoder = new TextDecoder();
        }
        async init() {
            if ('serial' in navigator) {
                try {
                    const port = await navigator.serial.requestPort();
                    await port.open({ baudRate: 9600 });
                    this.reader = port.readable.getReader();
                    let signals = await port.getSignals();
                    console.log(signals);
                    writer = port.writable.getWriter();
                }
                catch (err) {
                    console.error('There was an error opening the serial port:', err);
                }
            }
            else {
                console.error('Web serial doest seem to be enabled in your browser. Try enabling it by visiting:');
                console.error('chrome://flags/#enable-experimental-web-platform-features');
                console.error('opera://flags/#enable-experimental-web-platform-features');
                console.error('edge://flags/#enable-experimental-web-platform-features');
            }
        }
        async escribe() {

            
            const data = new Uint8Array([parseInt(piopio)]); // hello
            //const data = new String([parseInt(piopio)]); // hello
            console.log("el data",data);
            await writer.write(data);


            // Allow the serial port to be closed later.
            //writer.releaseLock();
            return " enviado:"+data+" ";
        }
        async read() {
            
            try {
                const readerData = await this.reader.read();
                console.log("datos leidos:",readerData)
                return this.decoder.decode(readerData.value);
            }
            catch (err) {
                const errorMessage = `error reading data: ${err}`;
                console.error(errorMessage);
                return errorMessage;
            }
        }
    }

    const serialScaleController = new SerialScaleController();
    const connect = document.getElementById('connect-to-serial');
    const getSerialMessages = document.getElementById('get-serial-messages');
    const putSerialMessages = document.getElementById('put-serial-messages');
    
    connect.addEventListener('pointerdown', () => {
      serialScaleController.init();
    });

    getSerialMessages.addEventListener('pointerdown', async () => {
      getSerialMessage();
    });

    putSerialMessages.addEventListener('pointerdown', async () => {
      putSerialMessage();
    });

    async function getSerialMessage() {
      document.querySelector("#serial-messages-container .message").innerText += await serialScaleController.read()
    }

    async function putSerialMessage() {
      document.querySelector("#serial-messages-container .message").innerText += await serialScaleController.escribe()
    }
var piopio = ''; 
function capturanumero() {

fetch(' ''' 
        htttp = htttp + direcsaund + ''' ')
  .then(response => response.text())  .then((data) => {
    //console.log("paso el coso:",data)
    piopio = data
  });
  console.log("estamos en esta configuracion, la numero ",piopio);
  
  putSerialMessage();
  getSerialMessage();

  }


    setInterval('capturanumero()',2000);


  </script>
</body>
</html>'''
        return htttp
    index.exposed = True

#
if __name__ == '__main__':
    #elpulento = eval_js('google.colab.kernel.proxyPort(8888)')
    config = {'server.socket_host': '0.0.0.0','server.socket_port' : int(sys.argv[1])}
    cherrypy.config.update(config)
    cherrypy.quickstart(HelloWorld())

In [ ]:
import cherrypy

class HelloWorld(object):
    @cherrypy.expose
    def index(self):
        return 'Hello World!'

    @cherrypy.expose
    def greet(self, name):
        return 'Hello {}!'.format(name)

cherrypy.quickstart(HelloWorld())